# 🏠 DeepRoof-2026: Master Training Lab

### 🛠 Step 1: System-Level Environment Initialization
This cell handles **MMCV Source Compilation** (for Torch 2.4+), **MMSegmentation Repair**, and **CUDA Linking** directly in the system environment.

In [ ]:
import os
import sys
import subprocess
import torch
import ctypes
import glob
import shutil
from pathlib import Path

print('🛰 Initializing DeepRepair Protocol V6 (Comprehensive)....')

# --- 1. PATH RESOLUTION (NO VENV) ---
project_root = Path('/workspace/roof')
if not project_root.exists():
    project_root = Path(os.getcwd()).parent

# Add project root to sys.path if not present
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    
print(f'📂 Project Root: {project_root}')
print(f'🐍 Python: {sys.executable}')

# --- 2. NUCLEAR CUDA LINKER (System Scan) ---
def nuclear_cuda_fix():
    if not torch.cuda.is_available():
        print('❄️ CPU Mode: Skipping CUDA linking.')
        return True

    print('🔍 Searching for libcudart.so...')
    # Common locations in containers
    search_patterns = [
        '/usr/local/cuda*/lib64/libcudart.so*',
        '/usr/lib/x86_64-linux-gnu/libcudart.so*',
        '/usr/lib/libcudart.so*'
    ]
    
    found_lib = None
    for pattern in search_patterns:
        matches = glob.glob(pattern)
        if matches:
            # Prefer specific version if multiple
            found_lib = sorted(matches)[-1]
            break
            
    if found_lib:
        print(f'📍 Found linking target: {found_lib}')
        try:
            ctypes.CDLL(found_lib, mode=ctypes.RTLD_GLOBAL)
            print('✅ CUDA Runtime force-loaded.')
        except Exception as e:
            print(f'⚠️ Force-load warning: {e}')
    else:
        print('⚠️ Could not find libcudart.so in standard paths. Assuming built-in.')
    return True

# --- 3. MMCV SOURCE COMPILER (Torch 2.4+) ---
def match_mmcv_to_torch():
    torch_ver = torch.__version__
    print(f'🔍 Detected: Torch {torch_ver}')
    
    mmcv_ok = False
    try:
        import mmcv
        from mmcv.ops import point_sample
        print('✅ MMCV is fully functional.')
        mmcv_ok = True
    except (ImportError, ModuleNotFoundError) as e:
        print(f'❌ MMCV Error: {e}')
    except Exception as e:
        print(f'❌ Unknown MMCV Error: {e}')

    if mmcv_ok: return True

    # Logic determines remediation
    print('🔄 Attempting Repair...')

    # Check for bleeding edge torch
    is_bleeding_edge = False
    if '+' in torch_ver: 
        base_ver = torch_ver.split('+')[0]
    else:
        base_ver = torch_ver
        
    major, minor = map(int, base_ver.split('.')[:2])
    if major >= 2 and minor >= 4:
        is_bleeding_edge = True

    if is_bleeding_edge:
        print('⚠️ Bleeding-edge Torch (>=2.4) detected. BINARY WHEELS DO NOT EXIST.')
        print('🛠 Starting SOURCE COMPILATION (approx 5-10 mins)...')
        
        # Cleaning old
        subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'mmcv'], check=False)
        
        mmcv_dir = project_root / 'mmcv-source'
        if mmcv_dir.exists(): shutil.rmtree(mmcv_dir)
        
        # Clone
        subprocess.check_call(['git', 'clone', '-b', 'v2.2.0', 'https://github.com/open-mmlab/mmcv.git', str(mmcv_dir)])
        
        # Compile
        env = os.environ.copy()
        env['MMCV_WITH_OPS'] = '1'
        env['FORCE_CUDA'] = '1'
        env['MAX_JOBS'] = '8'
        
        print('⏳ Compiling... check terminal for details if stuck.')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '.'], cwd=str(mmcv_dir), env=env)
        print('✅ Compilation Complete.')
        shutil.rmtree(mmcv_dir)
        return False
    else:
         print('ℹ️ Standard Torch detected. Trying MIM install.')
         subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U', 'openmim'])
         subprocess.check_call([sys.executable, '-m', 'mim', 'install', 'mmcv>=2.0.0'])
         return False

# --- 4. MMSEGMENTATION CHECK & REPAIR ---
def check_mmseg():
    print('🔍 Checking MMSegmentation installation...')
    try:
        import mmseg
        print(f'   MMSeg version: {mmseg.__version__}')
        try:
            from mmseg.models.segmentors.mask2former import Mask2Former  # noqa: F401
            print('✅ Mask2Former segmentor is available.')
            return True
        except ImportError:
            try:
                from mmseg.models.segmentors import Mask2Former  # noqa: F401
                print('✅ Mask2Former segmentor is available.')
                return True
            except ImportError:
                from mmseg.models.segmentors import EncoderDecoder  # noqa: F401
                from mmseg.models.decode_heads import Mask2FormerHead  # noqa: F401
                print('ℹ️ Mask2Former segmentor not exported by this mmseg build; using EncoderDecoder compatibility path.')
                return True

    except (ImportError, ModuleNotFoundError) as e:
        print(f'❌ MMSegmentation Issue: {e}')
        print('🔄 Reinstalling MMSegmentation via MIM...')
        
        subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'mmsegmentation'], check=False)
        subprocess.check_call([sys.executable, '-m', 'mim', 'install', 'mmsegmentation>=1.2.2'])
        print('✅ Reinstall complete. PLEASE RESTART KERNEL.')
        return False
    except Exception as e:
        print(f'⚠️ Unknown MMSeg error: {e}')
        return False
    return True

# --- 5. SAFETY NOTE ---
def patch_assertions():
    print('ℹ️ Skipping site-packages patching for mmseg/__init__.py (safer and reproducible).')
    print('   If mmcv/mmseg versions are incompatible, reinstall matching versions instead of editing package files.')
    return True

if nuclear_cuda_fix() and match_mmcv_to_torch():
    if check_mmseg():
        patch_assertions()
        print('🚀 System Ready.')
    else:
        print('\n⚠️  MMSEG UPDATED. PLEASE RESTART KERNEL.')
else:
    print('\n⚠️  ENVIRONMENT UPDATED. PLEASE RESTART KERNEL.')

## 📂 1. Dataset Preview

Visualize the **OmniCity** satellite imagery and ground truth **Masks** + **Surface Normals**.

In [ ]:
def preview_dataset(num_samples=3):
    import matplotlib.pyplot as plt
    import numpy as np
    import cv2
    
    data_path = project_root / 'data/OmniCity'
    train_file = data_path / 'train.txt'
    
    if not train_file.exists():
        print(f'❌ Multi-task training data not found at {data_path}.')
        return
        
    with open(train_file, 'r') as f:
        sample_ids = [line.strip() for line in f.readlines()[:num_samples]]
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))
    for i, sid in enumerate(sample_ids):
        img = cv2.cvtColor(cv2.imread(str(data_path / 'images' / (sid + '.jpg'))), cv2.COLOR_BGR2RGB)
        
        mask = cv2.imread(str(data_path / 'masks' / (sid + '.png')), cv2.IMREAD_UNCHANGED)
        mask_vis = cv2.applyColorMap(((mask % 20) * 12).astype(np.uint8), cv2.COLORMAP_JET)
        
        axes[i, 0].imshow(img); axes[i, 0].set_title(sid); axes[i, 0].axis('off')
        axes[i, 1].imshow(mask_vis); axes[i, 1].set_title('Mask'); axes[i, 1].axis('off')
        
        norm_path = data_path / 'normals' / (sid + '.npy')
        if norm_path.exists():
            normals = np.load(str(norm_path))
            axes[i, 2].imshow(((normals + 1) * 127.5).astype(np.uint8))
        axes[i, 2].set_title('Normals'); axes[i, 2].axis('off')
        
    plt.tight_layout(); plt.show()

preview_dataset(num_samples=2)

## 🚀 2. Absolute Ideal SOTA Training Configuration

Resuming from `iter_40000.pth` with **Physical Consistency Refinements**:
- **Config**: `deeproof_production_swin_L.py` (300 Queries, MSTrain, ShadowAug)
- **Geometry Loss**: `PhysicallyWeightedNormalLoss` (Azimuth Stability)
- **Vectorization**: Global Dominant Orientation Snapping (CAD-ready)
- **Load From**: `iter_40000.pth` (Baseline weights)

In [ ]:
from mmengine.config import Config
from mmseg.utils import register_all_modules

# Ensure mmseg registries/default scope are available in this kernel.
register_all_modules(init_default_scope=False)

# Explicitly import custom modules so they are registered before Runner build.
import deeproof.models.backbones.swin_v2_compat
import deeproof.models.deeproof_model
import deeproof.models.heads.mask2former_head
import deeproof.models.heads.geometry_head
import deeproof.models.losses
# FIX: Must import dataset module so DeepRoofDataset is registered in DATASETS registry
# Without this, Runner.build() raises KeyError: DeepRoofDataset not found
import deeproof.datasets.roof_dataset

CONFIG_PATH = str(project_root / 'configs/deeproof_production_swin_L.py')
# FIX: Updated work dir to v3 (fresh run with all bug fixes applied)
WORK_DIR = str(project_root / 'work_dirs/deeproof_absolute_ideal_v1')

cfg = Config.fromfile(CONFIG_PATH)
cfg.default_scope = 'mmseg'

# Safety: keep Swin window size compatible with loaded pretrained weights.
if cfg.model.get('backbone', {}).get('window_size', None) not in (None, 12):
    cfg.model.backbone.window_size = 12

# Safety: SegDataPreProcessor.train needs exactly one of size/size_divisor.
dp = cfg.model.setdefault('data_preprocessor', {})
has_size = dp.get('size') is not None
has_div = dp.get('size_divisor') is not None
if has_size and has_div:
    dp.pop('size', None)
elif (not has_size) and (not has_div):
    dp['size_divisor'] = 32

# Safety: normalize loop keys if a stale/partially-merged config is loaded.
if cfg.train_cfg.get('type') == 'EpochBasedTrainLoop' and 'max_iters' in cfg.train_cfg:
    cfg.train_cfg.pop('max_iters')
if cfg.train_cfg.get('type') == 'IterBasedTrainLoop' and 'max_epochs' in cfg.train_cfg:
    cfg.train_cfg.pop('max_epochs')

# Safety: mmengine PolyLR expects eta_min in this runtime.
for sch in cfg.get('param_scheduler', []):
    if isinstance(sch, dict) and sch.get('type') == 'PolyLR' and 'min_lr' in sch and 'eta_min' not in sch:
        sch['eta_min'] = sch.pop('min_lr')

# Safety: load_from should point to best iter_5000 checkpoint from previous run.
# Update this path if your best checkpoint has a different location.
import os
best_ckpt_candidates = [
    str(project_root / 'work_dirs/swin_l_finetune_v2/best_mIoU_iter_5000.pth'),
    str(project_root / 'work_dirs/swin_l_scratch_v1/iter_40000.pth'),
]
for ckpt in best_ckpt_candidates:
    if os.path.exists(ckpt):
        # cfg.load_from = 
cfg.load_from = '/workspace/roof/work_dirs/swin_l_finetune_v3/best_mIoU_iter_25000.pth'ckpt
        print(f'Checkpoint found: {ckpt}')
        break
else:
    print('WARNING: No checkpoint found! Training will start from random init.')

cfg.work_dir = WORK_DIR
cfg.data_root = str(project_root / 'data/OmniCity/')
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_root = cfg.data_root

loop_type = cfg.train_cfg.get('type', 'UnknownLoop')
if loop_type == 'IterBasedTrainLoop':
    train_span = f"{cfg.train_cfg.max_iters} iters"
    report_interval = f"Every {cfg.train_cfg.get('val_interval', 'N/A')} iters"
else:
    train_span = f"{cfg.train_cfg.max_epochs} epochs"
    report_interval = f"Every {cfg.train_cfg.get('val_interval', 'N/A')} epoch(s)"

# Extract LR from optim_wrapper (MMEngine uses optim_wrapper, not bare optimizer)
lr = cfg.optim_wrapper.optimizer.lr

cls_w = cfg.model.decode_head.loss_cls.class_weight
geo_w = cfg.model.geometry_loss_weight
clip = cfg.optim_wrapper.get('clip_grad', {}).get('max_norm', 'N/A')

print('SOTA CONFIG LOADED (Absolute Ideal - Physically Weighted Loss)')
print(f'Training Span:        {train_span}')
print(f'Initial LR:           {lr}')
print(f'Gradient Clip Norm:   {clip}  (was 0.01 -> now 1.0)')
print(f'Validation Interval:  {report_interval}')
print(f'Batch Size:           {cfg.train_dataloader.batch_size}')
print(f'Load From:            {cfg.get("load_from", "None")}')
print(f'Work Dir:             {WORK_DIR}')
print(f'Class Weights:        bg={cls_w[0]}, flat={cls_w[1]}, sloped={cls_w[2]}, no_obj={cls_w[3]}')
print(f'Geometry Loss Weight: {geo_w}  (was 10.0 -> now 2.0)')


## 🚀 3. Kickoff Training

This will invoke the `mmengine.Runner` and begin fine-tuning from `iter_40000.pth`.

In [ ]:
import torch
from mmengine.runner import Runner

device_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'
print(f'Starting Fine-Tune on: {device_name}')
print(f'Batch size:            {cfg.train_dataloader.batch_size}')
print(f'Work dir:              {cfg.work_dir}')
print(f'Sloped class weight:   {cfg.model.decode_head.loss_cls.class_weight[2]}x')
print(f'Geometry loss weight:  {cfg.model.geometry_loss_weight}')
print(f'Gradient clip norm:    {cfg.optim_wrapper.get("clip_grad", {}).get("max_norm", "N/A")}')
print()

runner = Runner.from_cfg(cfg)
runner.train()


## 📊 4. Monitoring & Metrics

Run this cell during or after training to visualize performance trends.

In [ ]:
import json
import os
import glob
import matplotlib.pyplot as plt

def plot_training_logs(log_path):
    """Plot all losses including loss_geometry and mIoU from MMEngine JSON log."""
    if not os.path.exists(log_path):
        print('No logs found yet.')
        return

    # Separate tracking for train metrics and val metrics
    train_data = {'iter': [], 'loss': [], 'loss_cls': [], 'loss_mask': [], 'loss_dice': [], 'loss_geometry': []}
    val_data = {'iter': [], 'mIoU': []}

    with open(log_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                data = json.loads(line)
            except json.JSONDecodeError:
                continue

            # Training step metrics
            if 'loss' in data and 'mode' not in data:
                train_data['iter'].append(data.get('iter', 0))
                train_data['loss'].append(data.get('loss', 0))
                for k in ['loss_cls', 'loss_mask', 'loss_dice', 'loss_geometry']:
                    if k in data:
                        train_data[k].append(data[k])
            # Validation metrics (mode=val)
            if data.get('mode') == 'val' or 'mIoU' in data:
                val_data['iter'].append(data.get('iter', data.get('step', 0)))
                val_data['mIoU'].append(data.get('mIoU', 0))

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Plot 1: Total loss
    if train_data['iter']:
        axes[0].plot(train_data['iter'], train_data['loss'], color='#e74c3c', linewidth=1.5, label='Total Loss')
        axes[0].set_title('Total Loss')
        axes[0].set_xlabel('Iteration')
        axes[0].set_ylabel('Loss')
        axes[0].legend()
        axes[0].grid(alpha=0.3)

    # Plot 2: Individual losses
    colors = {'loss_cls': '#3498db', 'loss_mask': '#2ecc71', 'loss_dice': '#f39c12', 'loss_geometry': '#9b59b6'}
    for k, color in colors.items():
        if train_data[k]:
            axes[1].plot(train_data['iter'][:len(train_data[k])], train_data[k], color=color, linewidth=1.5, label=k, alpha=0.8)
    axes[1].set_title('Individual Losses')
    axes[1].set_xlabel('Iteration')
    axes[1].set_ylabel('Loss')
    axes[1].legend(fontsize=8)
    axes[1].grid(alpha=0.3)

    # Plot 3: mIoU
    if val_data['iter']:
        axes[2].plot(val_data['iter'], val_data['mIoU'], color='#27ae60', marker='o', linewidth=2, label='mIoU')
        best_idx = val_data['mIoU'].index(max(val_data['mIoU']))
        axes[2].axvline(val_data['iter'][best_idx], color='red', linestyle='--', alpha=0.7,
                        label=f"Best: {val_data['mIoU'][best_idx]:.3f} @ iter {val_data['iter'][best_idx]}")
        axes[2].set_title('Validation mIoU')
        axes[2].set_xlabel('Iteration')
        axes[2].set_ylabel('mIoU')
        axes[2].legend()
        axes[2].grid(alpha=0.3)
    else:
        axes[2].text(0.5, 0.5, 'No validation data yet', ha='center', va='center', transform=axes[2].transAxes)

    plt.suptitle(f'Training Monitor: {os.path.basename(log_path)}', fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(WORK_DIR, 'training_monitor.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print(f'Plot saved to {WORK_DIR}/training_monitor.png')

# FIX: MMEngine logs are in {work_dir}/TIMESTAMP/vis_data/scalars.json
# Try both standard paths
log_candidates = (
    glob.glob(os.path.join(WORK_DIR, '*/vis_data/scalars.json')) +
    glob.glob(os.path.join(WORK_DIR, 'vis_data/scalars.json'))
)
if log_candidates:
    log_path = sorted(log_candidates)[-1]  # Most recent
    print(f'Reading log: {log_path}')
    plot_training_logs(log_path)
else:
    print('No logs found yet - run training first.')
    print(f'Expected location: {WORK_DIR}/TIMESTAMP/vis_data/scalars.json')
